In [1]:
import glob
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
import os
import time
from urllib.parse import urlparse
import random

all_txt_files = glob.glob(os.getcwd()+'\\prod_urls\\*.txt')

In [2]:
home_dir = os.getcwd()
results_dir = home_dir + '\\results'
images_dir = results_dir + '\\images'
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

In [3]:
all_txt_files

['C:\\Users\\Administrator\\webscraping-product-details-flipkart\\prod_urls\\Literature Books.txt',
 'C:\\Users\\Administrator\\webscraping-product-details-flipkart\\prod_urls\\Self-Help Books.txt']

In [4]:
genres_and_links = dict()

for filename in all_txt_files:
    with open(filename,'r') as file:
        relative_filename = filename.split('\\')[-1]
        genres_and_links[relative_filename.replace('.txt','')] = file.readlines()

In [5]:
for kk,vv in genres_and_links.items():
    print(len(vv),'-',kk)

2000 - Literature Books
2000 - Self-Help Books


In [6]:
dumb_urls = ['https://www.thepythoncode.com/','https://www.geeksforgeeks.org/','https://pynative.com/','http://facebook.com/','https://www.amazon.in/','https://www.tutorialspoint.com/','https://docs.python.org/','https://chromedriver.chromium.org/','https://twitter.com/']

In [7]:
def replace_multispaces_with_single_space(text):
    text = re.sub('\\s+',' ',text)
    return text

In [8]:
ultimate_dataframes = dict()

In [9]:
driver = None

for genre, links in genres_and_links.items():
    
    results_dict = dict()

    list_of_book_titles = list()
    list_of_prices = list()
    list_of_publishers = list()
    list_of_pages = list()
    list_of_isbns = list()
    list_of_genres = list()
    list_of_ratings = list()
    list_of_authors = list()
    list_of_book_summaries = list()
    list_of_image_names = list()
    list_of_language= list()

    main_counter = 1
    for link in links:
        
        requests.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
        #time.sleep(1)
        resp = requests.get(link)
        
        soup = BeautifulSoup(resp.content,'html.parser')
        
        book_title = 'NA'
        try:
            book_title_ele = soup.find('span',{'class':'_35KyD6'})
            book_title = book_title_ele.get_text()
            book_title = replace_multispaces_with_single_space(book_title)
        except:
            requests.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
            #time.sleep(1)
            resp = requests.get(link)
            soup = BeautifulSoup(resp.content,'html.parser')
            book_title_ele = soup.find('span',{'class':'_35KyD6'})
            book_title = book_title_ele.get_text()
            book_title = replace_multispaces_with_single_space(book_title)
        
        price = 'NA'
        try:
            price = soup.find('div',{'class':'_1vC4OE _3qQ9m1'}).get_text()
            price = re.sub('₹','',price)
        except:
            requests.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
            #time.sleep(1)
            resp = requests.get(link)
            soup = BeautifulSoup(resp.content,'html.parser')
            price = soup.find('div',{'class':'_1vC4OE _3qQ9m1'}).get_text()
            price = re.sub('₹','',price)
            
        spec_tbody_elements = soup.find_all('tr',{'class':'_3_6Uyw row'})
        publisher = 'NA'
        for tbody in spec_tbody_elements:
            tr_elements = tbody.find_all('td')
            if len(tr_elements)==2:
                if tr_elements[0] and tr_elements[0].text=='Imprint':
                    publisher = tr_elements[1].get_text()
    
        no_of_pages = 'NA'
        isbn = 'NA'
        language = 'NA'
        highlights_li_elements = soup.find_all('li',{'class':'_2-riNZ'})
        for li in highlights_li_elements:
            if li.get_text() and li.get_text().startswith('Pages'):
                no_of_pages = li.get_text().split(': ')[1]
            if li.get_text() and li.get_text().startswith('ISBN'):
                isbn = li.get_text().split(': ')[1]
            if li.get_text() and li.get_text().startswith('Language'):
                language = li.get_text().split(': ')[1]
        
        rating = 'NA'
        rating_parent_element = soup.find('div',{'class':'_3ors59'})
        if rating_parent_element:
            rating_element = rating_parent_element.find('div',{'class':'hGSR34'})
            if rating_element:
                rating = rating_element.get_text()
        
        author = 'NA'
        book_summary = 'NA'
        book_desc_list = list()
        author_description_elements = soup.find_all('div',{'class':'_1y9a40'})
        for element in author_description_elements:
            if element.find('div',{'class':'_1oCqc9'}) and element.find('div',{'class':'_1oCqc9'}).get_text()=='Author':
                author = element.find('div',{'class':'_3cpW1u'}).get_text().strip()
            if element.find('div',{'class':'_1oCqc9'}) and element.find('div',{'class':'_1oCqc9'}).get_text()=='Description':
                if element.find('div',{'class':'_3cpW1u'}):
                    book_summary = element.find('div',{'class':'_3cpW1u'}).get_text()
                    book_summary = replace_multispaces_with_single_space(book_summary)
                #desc_p_elements = element.find_all('p');
                #book_desc_list = [re.sub('\\s+',' ',p.get_text()) for p in desc_p_elements if p]
                #book_summary = ''.join(book_desc_list)
                
        
        image_urls = set()
        image_elements = soup.find_all('div',{'class':'_2_AcLJ'})
        image_url = None
        #for ele in image_elements:
            #if ele.has_attr('style'):
                #if '(' in ele.get('style') and ')' in ele.get('style'):
                    #image_url = ele.get('style').split('(')[1].split(')')[0]
                #if image_url is not None:
                    #image_url = re.sub('\\w+?q=\\d+','',image_url)
                    #image_url = re.sub('https://rukminim1.flixcart.com/image/\\d+/\\d+/','https://rukminim1.flixcart.com/image/512/512/',image_url)
                    #url_parse_results = urlparse(image_url)
                    #if all([url_parse_results.scheme, url_parse_results.netloc, url_parse_results.path]):
                        #image_urls.add(image_url)
        
        i_file_name = results_dir + '\\images\\' + genre + '\\' +str(main_counter)+'.jpeg'
        if not os.path.isfile(i_file_name):
            if driver is None:
                #options = webdriver.ChromeOptions()
                #options.add_argument("--start-maximized")
                #driver = webdriver.Chrome(executable_path="chromedriver.exe",options=options)
                driver = webdriver.Firefox(executable_path='geckodriver.exe')
            driver.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
            #time.sleep(1)
            driver.get(link)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source,'html.parser')
            image_divs = soup.find_all('div',{'class':'_1ov7-N'})
            for image_div in image_divs:
                img_elements = image_div.find_all('img')
                for img in img_elements:
                    if img.has_attr('src'):
                        image_url = img.get('src')
            if image_url is not None:
                image_url = re.sub('\\w+?q=\\d+','',image_url)
                image_url = re.sub('https://rukminim1.flixcart.com/image/\\d+/\\d+/','https://rukminim1.flixcart.com/image/512/512/',image_url)
                url_parse_results = urlparse(image_url)
                if all([url_parse_results.scheme, url_parse_results.netloc, url_parse_results.path]):
                    image_urls.add(image_url)
        
        if len(image_urls)==0:
            image_url = 'https://img1a.flixcart.com/www/linchpin/fk-cp-zion/img/flipkart-plus_4ee2f9.png'
            image_urls.add(image_url)
            
        #i_counter = 1
        i_names = list()
        for i_url in image_urls:
            genre_images_dir = results_dir + '\\images\\' + genre
            if not os.path.exists(genre_images_dir):
                os.makedirs(genre_images_dir)
            #i_name = genre_images_dir + '\\' +str(main_counter+1)+'.'+str(i_counter)+'.jpeg'
            i_name = genre_images_dir + '\\' +str(main_counter)+'.jpeg'
            
            if not os.path.isfile(i_name):
                requests.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
                #time.sleep(1)
                i_response = requests.get(i_url)
                with open(i_name, 'wb') as f:
                    f.write(i_response.content)
            i_name = i_name.split('\\')[-1]
            i_names.append(i_name)
            #i_counter +=1
            break;
        

        #print(book_title)#'title_bindtype_language-'
        #print(price)
        #print(publisher)
        #print(no_of_pages)
        #print(isbn)
        #print(genre)
        #print(rating)
        #print(author)
        #print(book_summary)
        #print(image_urls)
        #print(language)
        
        list_of_book_titles.append(book_title)
        list_of_prices.append(price)
        list_of_publishers.append(publisher)
        list_of_pages.append(no_of_pages)
        list_of_isbns.append(isbn)
        list_of_genres.append(genre.replace(' Books',''))
        list_of_ratings.append(rating)
        list_of_authors.append(author)
        list_of_book_summaries.append(book_summary)
        list_of_image_names.append(', '.join(i_names))
        list_of_language.append(language)
        
        #print('-----------------')
        
        if not os.path.exists(home_dir+'\\logs'):
            os.makedirs(home_dir+'\\logs')
        
        with open(home_dir+'\\logs\\'+genre+'_audit.log','a+') as file:
            file.write(str(main_counter)+os.linesep)
        
        #if main_counter==3:
            #break;
        main_counter += 1
    
    results_dict['title'] = list_of_book_titles
    results_dict['price'] = list_of_prices
    results_dict['publishers'] = list_of_publishers
    results_dict['number_of_pages'] = list_of_pages
    results_dict['isbn_10'] = list_of_isbns
    results_dict['isbn_13'] = list_of_isbns
    results_dict['genres'] = list_of_genres
    results_dict['rating'] = list_of_ratings
    results_dict['authors'] = list_of_authors
    results_dict['description'] = list_of_book_summaries
    results_dict['image'] = list_of_image_names
    results_dict['thumb'] = list_of_image_names
    results_dict['languages'] = list_of_language
    
    df = pd.DataFrame(results_dict)
    df = df[['isbn_10','isbn_13','image','thumb','title','authors','languages','genres','number_of_pages','description','publishers','rating','price']]
    export_file_name = results_dir + '\\' + genre + '.csv'
    df.to_csv(export_file_name,index=False)
    ultimate_dataframes[genre] = df
    print('yahooooooo!!! Completed for','\"',genre,'\"')
    
    driver.quit()
    driver = None
    

WebDriverException: Message: unknown error: cannot find Chrome binary


In [ ]:
if driver is not None:
    driver.quit()

In [ ]:
print('Everything completed. Thank god.')